In [1]:
Installations
%pip install google-api-python-client
%pip install pyodbc

In [2]:
# Importing Libraries
from dotenv import dotenv_values
from googleapiclient.discovery import build
from docx import Document

In [3]:
# Installations
# %pip install google-api-python-client
# %pip install pyodbc
# Importing Libraries
from dotenv import dotenv_values
from googleapiclient.discovery import build
from docx import Document

# Load environment variables from the .env file into a dictionary
environment_vars = dotenv_values('.env')

# Get the credential values from the '.env' file 
my_api_key = environment_vars.get("API_KEY")

# Enter your API key here
api_key = my_api_key

# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

# Get the playlist URL from the user
playlist_url = input("Enter the playlist URL: ")

# Extract the playlist ID from the URL
playlist_id = playlist_url.split("list=")[1]

# Get repetitive terms from the user
repetitive_terms_input = input("Enter repetitive terms (comma-separated): ")
repetitive_terms = [term.strip().upper() for term in repetitive_terms_input.split(',')]

# Define the headings dictionary
headings = {}

# Get all the video details from the playlist
videos = []
next_page_token = None

while True:
    # Get the playlist items
    playlist_items = youtube.playlistItems().list(
        part="contentDetails",
        playlistId=playlist_id,
        maxResults=50,
        pageToken=next_page_token
    ).execute()

    # Get the video IDs
    video_ids = [item['contentDetails']['videoId'] for item in playlist_items['items']]

    # Get the video details
    video_details = youtube.videos().list(
        part="snippet",
        id=','.join(video_ids)
    ).execute()

    for item in video_details['items']:
        title = item['snippet']['title']
        video_id = item['id']
        video_url = f"https://www.youtube.com/watch?v={video_id}"

        # Group videos based on the repetitive terms
        found = False
        for term in repetitive_terms:
            if term in title.upper():
                if term not in headings:
                    headings[term] = chr(len(headings) + 65)
                videos.append({"title": title, "url": video_url, "group": headings[term]})
                found = True
                break

        # Add videos without repetitive terms to Introduction or Conclusion
        if not found:
            if item == video_details['items'][0]:
                videos.append({"title": title, "url": video_url, "group": "Introduction"})
            elif item == video_details['items'][-1]:
                videos.append({"title": title, "url": video_url, "group": "Conclusion"})

    # Check if there are more pages
    next_page_token = playlist_items.get('nextPageToken')

    if not next_page_token:
        break

# Create a Word document
doc = Document()

# Print the video titles and URLs grouped by the headings
for group in sorted(headings.values()):
    doc.add_paragraph(f"\n{group}. {list(headings.keys())[list(headings.values()).index(group)]}")
    count = 1
    for video in videos:
        if video['group'] == group:
            doc.add_paragraph(f"{group}.{count} {video['title']} - {video['url']}")
            count += 1

# Print videos without repetitive terms under Introduction or Conclusion
for group in ["Introduction", "Conclusion"]:
    doc.add_paragraph(f"\n{group}")
    count = 1
    for video in videos:
        if video['group'] == group:
            doc.add_paragraph(f"{group}.{count} {video['title']} - {video['url']}")
            count += 1

Enter the playlist URL: https://www.youtube.com/watch?v=rGx1QNdYzvs&list=PLUaB-1hjhk8FE_XZ87vPPSfHqb6OcM0cF
Enter repetitive terms (comma-separated): EXCEL, POWER BI, TABLEAU, SQL, PYTHON


In [4]:
# Save the Word document
doc.save("organized_playlist.docx")
print("Output saved in organized_playlist.docx")

Output saved in playlist_output.docx
